# LIBRARIES

In [3]:
import pandas as pd
import numpy as np

#googlee play scrapper
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all

#cleaning
import contractions

#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#feature extraction
from keybert import KeyBERT
kw_model = KeyBERT()
from keyphrase_vectorizers import KeyphraseCountVectorizer

#language id
from langdetect import detect

# SCRAP

In [4]:
reviews = reviews_all(
    'com.alodokter.android',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='id', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

alodokterraw = pd.DataFrame(np.array(reviews),columns=['review'])
alodokterraw = alodokterraw.join(pd.DataFrame(alodokterraw.pop('review').tolist()))
alodokterraw.to_csv('data_alodokterraw.csv',index=False)
alodokterraw

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,35a889f1-798b-4a71-bf8e-83425557ad93,Rizki Wardani,https://play-lh.googleusercontent.com/a-/ACNPE...,Parah nya resep obat ditebus dari tempat yg lu...,3,0,4.9.0,2022-11-13 12:22:11,None,NaT
1,24ce8dd1-9a98-428a-b616-28c290fe7d45,FediOnARy Ndoys,https://play-lh.googleusercontent.com/a-/ACNPE...,Sudah lama pake aplikasi ini utk keluarga kami...,5,0,4.9.0,2022-11-13 07:24:31,None,NaT
2,d255e8fa-5abf-47f9-b320-60629834269b,Amara Widiyanty,https://play-lh.googleusercontent.com/a-/ACNPE...,Slow response,2,1,4.9.0,2022-11-13 06:47:55,None,NaT
3,8f581c5a-717b-4cf0-ba50-9f3ed683dfdb,vb nezz,https://play-lh.googleusercontent.com/a-/ACNPE...,Very helpfull,5,0,4.9.0,2022-11-13 02:07:09,None,NaT
4,310ee464-6e9b-4f24-b413-0d7e3a26741a,Heru Permana,https://play-lh.googleusercontent.com/a-/ACNPE...,Very Good,5,0,None,2022-11-12 20:48:52,None,NaT
...,...,...,...,...,...,...,...,...,...,...
18366,f6f1fbd8-c7e7-4533-af46-91a47ff972e3,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"I love it, this app makes me easier to consult...",5,2,None,2016-03-15 15:15:36,None,NaT
18367,041f19c5-4ab9-49ef-af67-6538e5f9b7b6,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Finally alodokter launch the alodokter apps in...,5,0,1.0.0,2016-03-14 19:25:34,None,NaT
18368,309e6e74-ad6a-4660-8dfe-bb4bf297a999,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Sangat berguna untuk zaman sekarang yang diman...,5,0,None,2016-03-14 10:53:01,None,NaT
18369,c567eac0-6b7c-4d6c-a1f8-9b369618970e,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Berguna untuk menjawab pertanyaan saya tentang...,5,1,None,2016-03-14 10:36:42,None,NaT


# DROP COLUMNS & FILTER DATE

In [5]:
alodokterfiltered = pd.read_csv(r'data_alodokterraw.csv')
alodokterfiltered['at'] = pd.to_datetime(alodokterfiltered['at'], format='%Y-%m-%d')
alodokterfiltered = alodokterfiltered.loc[(alodokterfiltered['at'] >= "2022-1-1") & (alodokterfiltered['at'] <= "2022-10-31")]
alodokterfiltered = alodokterfiltered.reset_index(drop=True)
alodokterfiltered = alodokterfiltered.drop(columns=['reviewId','userName','userImage','thumbsUpCount','reviewCreatedVersion','at','replyContent','repliedAt'])
alodokterfiltered.to_csv('data_alodokterfiltered.csv',index=False)
alodokterfiltered

,content,score
0,Lebih bagus dari H@lodoc ...... dokternya juga...,5
1,The app is good. This 1 star is for your Whats...,1
2,Dokter Arief Tri Bowo nya sabar...👍👍👍,5
3,Cepat disaat butuh. Untuk sakit2 umum bs bgt d...,5
4,Ini mstnya bs konsultasi ynya jawab tp kota ma...,1
...,...,...
777,Mau vaksin Sinovac,4
778,Helpful with fair price,5
779,fast response,5
780,Bagus sangat membantu,5


# CLEAN

In [6]:
alodoktercleaned = pd.read_csv(r'data_alodokterfiltered.csv')

#choose reviews with 2 words or more
alodoktercleaned = alodoktercleaned[[len(i.split())>2 for i in alodoktercleaned['content'].values]]

#fix contractions
alodoktercleaned['content_contractions'] = alodoktercleaned['content'].apply(lambda x: contractions.fix(x))

#translate the reviews to english
#alodoktercleaned['content_translated'] = alodoktercleaned['content_contractions'].apply(lambda x: gs.translate(x, 'en'))
alodoktercleaned.to_csv('data_alodoktercleaned.csv',index=False)
alodoktercleaned

,content,score,content_contractions
0,Lebih bagus dari H@lodoc ...... dokternya juga...,5,Lebih bagus dari H@lodoc ...... dokternya juga...
1,The app is good. This 1 star is for your Whats...,1,The app is good. This 1 star is for your Whats...
2,Dokter Arief Tri Bowo nya sabar...👍👍👍,5,Dokter Arief Tri Bowo nya sabar...👍👍👍
3,Cepat disaat butuh. Untuk sakit2 umum bs bgt d...,5,Cepat disaat butuh. Untuk sakit2 umum bs bgt d...
4,Ini mstnya bs konsultasi ynya jawab tp kota ma...,1,Ini mstnya bs konsultasi ynya jawab tp kota ma...
...,...,...,...
776,"Very good, fast response",5,"Very good, fast response"
777,Mau vaksin Sinovac,4,Mau vaksin Sinovac
778,Helpful with fair price,5,Helpful with fair price
780,Bagus sangat membantu,5,Bagus sangat membantu


In [10]:
alodoktercleaned['lang_id'] = alodoktercleaned['content_contractions'].apply(lambda x: detect(x))
alodokterfinal = alodoktercleaned.loc[alodoktercleaned['lang_id'] == "en"]
alodokterfinal = alodokterfinal.drop(columns=["content",'lang_id'])
alodokterfinal.to_csv('data_alodokterfinal.csv',index=False)
alodokterfinal

,score,content_contractions
1,1,The app is good. This 1 star is for your Whats...
11,5,Good application very helpful. Thanks Alodokter
13,5,Quick answer and helpful
46,1,Very bad apps
52,5,Easy to use
...,...,...
740,5,The best app
741,5,I got a fast response from the doctor
771,1,Useless and detrimental app! user money thief!...
776,5,"Very good, fast response"


# SENTIMENT ANALYSIS

In [12]:
alodoktervader = pd.read_csv(r'data_alodokterfinal.csv')
alodoktervader['compound'] = [analyzer.polarity_scores(x)['compound'] for x in alodoktervader['content_contractions']]
alodoktervader['neg'] = [analyzer.polarity_scores(x)['neg'] for x in alodoktervader['content_contractions']]
alodoktervader['neu'] = [analyzer.polarity_scores(x)['neu'] for x in alodoktervader['content_contractions']]
alodoktervader['pos'] = [analyzer.polarity_scores(x)['pos'] for x in alodoktervader['content_contractions']]


#convert rating
def getrating (compound,positive,negative):
    if compound > 0.001:
        if positive > 0.5:
            return 2
        else:
            return 1
    elif compound < -0.001:
        if negative > 0.5:
            return -2
        else:
            return -1
    elif (compound > -0.001) and (compound < 0.001):
        return 0
alodoktervader['vader_rating'] = alodoktervader.apply(lambda x: getrating(x.compound,x.pos,x.neg),axis=1)
alodoktervader.to_csv('data_alodoktervader.csv',index=False)
alodoktervader

,score,content_contractions,compound,neg,neu,pos,vader_rating
0,1,The app is good. This 1 star is for your Whats...,0.4404,0.000,0.892,0.108,1
1,5,Good application very helpful. Thanks Alodokter,0.8470,0.000,0.246,0.754,2
2,5,Quick answer and helpful,0.4215,0.000,0.517,0.483,1
3,1,Very bad apps,-0.5849,0.655,0.345,0.000,-2
4,5,Easy to use,0.4404,0.000,0.408,0.592,2
...,...,...,...,...,...,...,...
78,5,The best app,0.6369,0.000,0.323,0.677,2
79,5,I got a fast response from the doctor,0.0000,0.000,1.000,0.000,0
80,1,Useless and detrimental app! user money thief!...,-0.7772,0.263,0.737,0.000,-1
81,5,"Very good, fast response",0.4927,0.000,0.484,0.516,2


# KEYWORD EXTRACTION

In [13]:
doc = """
        The apps feel lights and fluid however the extra convenience fee doesnt make sense at all Way too expensive the more seats you want to order Very few promotions and the loyalty feature that still WIP Hope there are no ads that cover the entire screen for the next update Couldnt recommend to everyone for now
      """

In [14]:
from keybert import KeyBERT
kw_model = KeyBERT()

#he default pattern is <J.*>*<N.*>+ which means that it extract keyphrases that have 0 or more adjectives followed by 1 or more nouns.

from keyphrase_vectorizers import KeyphraseCountVectorizer
vectorizer = KeyphraseCountVectorizer(pos_pattern='<N.*>+<N.*>+|<N.*>+<N.*>+<N.*>+|<V.*>*<N.*>+|')
kw_model.extract_keywords(doc, vectorizer=vectorizer)

[('apps', 0.4264),
 ('convenience fee', 0.3447),
 ('loyalty feature', 0.3327),
 ('feel lights', 0.2818),
 ('promotions', 0.2791)]

In [15]:
alodokterkeybert = pd.read_csv(r'data_alodoktervader.csv')
alodokterkeybert['keyword'] = alodokterkeybert['content_contractions'].apply(lambda x: kw_model.extract_keywords(x, vectorizer=vectorizer))
alodokterkeybert.to_csv('data_alodokterkeybert.csv',index=False)
alodokterkeybert

,score,content_contractions,compound,neg,neu,pos,vader_rating,keyword
0,1,The app is good. This 1 star is for your Whats...,0.4404,0.000,0.892,0.108,1,"[(whatsapp cs, 0.7004), (app, 0.4297), (star, ..."
1,5,Good application very helpful. Thanks Alodokter,0.8470,0.000,0.246,0.754,2,"[(application, 0.5497), (thanks alodokter, 0.3..."
2,5,Quick answer and helpful,0.4215,0.000,0.517,0.483,1,"[(answer, 0.4)]"
3,1,Very bad apps,-0.5849,0.655,0.345,0.000,-2,"[(apps, 0.6496)]"
4,5,Easy to use,0.4404,0.000,0.408,0.592,2,[]
...,...,...,...,...,...,...,...,...
78,5,The best app,0.6369,0.000,0.323,0.677,2,"[(app, 0.7303)]"
79,5,I got a fast response from the doctor,0.0000,0.000,1.000,0.000,0,"[(doctor, 0.3393), (response, 0.2942)]"
80,1,Useless and detrimental app! user money thief!...,-0.7772,0.263,0.737,0.000,-1,"[(user money thief, 0.6705), (app, 0.5068), (u..."
81,5,"Very good, fast response",0.4927,0.000,0.484,0.516,2,"[(response, 0.5506)]"


In [208]:
data = []

#coba=pd.DataFrame(data)
a=-1
for i in features['keyword']:
    res = i.strip('][').split(', (')
    #print(i)
    a+=1
#    coba.insert(0,a)
    for j in range (len(res)):
        data.append({'nomer':a,'word':res[j]})
        #coba.append(res[j])
#pd.DataFrame(coba)
data=pd.DataFrame(data)
data=data.set_index('nomer')
data

,word
nomer,
0,"('whatsapp cs', 0.7004)"
0,"'app', 0.4297)"
0,"'star', 0.2012)"
0,"'response', 0.1543)"
0,"'wa', 0.1428)"
...,...
80,"'using user data', 0.4937)"
80,"'account', 0.3638)"
80,"'taking money', 0.3337)"


In [216]:
features = pd.read_csv(r'data_alodokterkeybert.csv')
features=pd.merge(features,data,left_index=True,right_index=True)
features.to_csv('features.csv',index=False)
features

,score,content_contractions,compound,neg,neu,pos,vader_rating,keyword,word
0,1,The app is good. This 1 star is for your Whats...,0.4404,0.000,0.892,0.108,1,"[('whatsapp cs', 0.7004), ('app', 0.4297), ('s...","('whatsapp cs', 0.7004)"
0,1,The app is good. This 1 star is for your Whats...,0.4404,0.000,0.892,0.108,1,"[('whatsapp cs', 0.7004), ('app', 0.4297), ('s...","'app', 0.4297)"
0,1,The app is good. This 1 star is for your Whats...,0.4404,0.000,0.892,0.108,1,"[('whatsapp cs', 0.7004), ('app', 0.4297), ('s...","'star', 0.2012)"
0,1,The app is good. This 1 star is for your Whats...,0.4404,0.000,0.892,0.108,1,"[('whatsapp cs', 0.7004), ('app', 0.4297), ('s...","'response', 0.1543)"
0,1,The app is good. This 1 star is for your Whats...,0.4404,0.000,0.892,0.108,1,"[('whatsapp cs', 0.7004), ('app', 0.4297), ('s...","'wa', 0.1428)"
...,...,...,...,...,...,...,...,...,...
80,1,Useless and detrimental app! user money thief!...,-0.7772,0.263,0.737,0.000,-1,"[('user money thief', 0.6705), ('app', 0.5068)...","'using user data', 0.4937)"
80,1,Useless and detrimental app! user money thief!...,-0.7772,0.263,0.737,0.000,-1,"[('user money thief', 0.6705), ('app', 0.5068)...","'account', 0.3638)"
80,1,Useless and detrimental app! user money thief!...,-0.7772,0.263,0.737,0.000,-1,"[('user money thief', 0.6705), ('app', 0.5068)...","'taking money', 0.3337)"
81,5,"Very good, fast response",0.4927,0.000,0.484,0.516,2,"[('response', 0.5506)]","('response', 0.5506)"


In [149]:
coba=[]
for i in features['keyword']:
    res = i.strip('][').split(', (')
    #print(i)
    for j in range (len(res)):
        #coba.append(res[j])
pd.DataFrame(coba)

,0
0,"('whatsapp cs', 0.7004)"
1,"'app', 0.4297)"
2,"'star', 0.2012)"
3,"'response', 0.1543)"
4,"'wa', 0.1428)"
...,...
185,"'using user data', 0.4937)"
186,"'account', 0.3638)"
187,"'taking money', 0.3337)"
188,"('response', 0.5506)"


# RE-SWOT

In [4]:
swot = pd.read_csv(r'features.csv')
swot

,score,review,compound,neg,neu,pos,vader_rating,keyword,word,feature
0,1,The app is good. This 1 star is for your Whats...,0.4404,0.000,0.892,0.108,1,"[('whatsapp cs', 0.7004), ('app', 0.4297), ('s...","('whatsapp cs', 0.7004)",customer service
1,1,The app is good. This 1 star is for your Whats...,0.4404,0.000,0.892,0.108,1,"[('whatsapp cs', 0.7004), ('app', 0.4297), ('s...","'response', 0.1543)",response
2,5,Good application very helpful. Thanks Alodokter,0.8470,0.000,0.246,0.754,2,"[('application', 0.5497), ('thanks alodokter',...","('application', 0.5497)",user experience
3,5,Quick answer and helpful,0.4215,0.000,0.517,0.483,1,"[('answer', 0.4)]","('answer', 0.4)",response
4,1,Very bad apps,-0.5849,0.655,0.345,0.000,-2,"[('apps', 0.6496)]","('apps', 0.6496)",user experience
...,...,...,...,...,...,...,...,...,...,...
114,1,Useless and detrimental app! user money thief!...,-0.7772,0.263,0.737,0.000,-1,"[('user money thief', 0.6705), ('app', 0.5068)...","('user money thief', 0.6705)",payment
115,1,Useless and detrimental app! user money thief!...,-0.7772,0.263,0.737,0.000,-1,"[('user money thief', 0.6705), ('app', 0.5068)...","'using user data', 0.4937)",data
116,1,Useless and detrimental app! user money thief!...,-0.7772,0.263,0.737,0.000,-1,"[('user money thief', 0.6705), ('app', 0.5068)...","'account', 0.3638)",account
117,5,"Very good, fast response",0.4927,0.000,0.484,0.516,2,"[('response', 0.5506)]","('response', 0.5506)",response


In [5]:
swot['feature'] = swot['feature'].replace(['customer experience'], 'user experience')
swot['feature'] = swot['feature'].replace(['reponse'], 'response')
feature_counted = swot['feature'].value_counts()
feature_counted = pd.DataFrame(feature_counted)
feature_counted.reset_index(inplace=True)
feature_counted = feature_counted.rename(columns={'index':'feature','feature':'count'})
feature_counted

,feature,count
0,user experience,39
1,doctor,17
2,response,16
3,consultation,10
4,information,6
5,customer service,4
6,payment,4
7,data,3
8,account,2
9,medicine,2


In [246]:
feature_counted['count'].sum()

119

In [6]:
trans=swot.groupby('feature')['vader_rating'].sum()
trans=pd.DataFrame(trans)
trans.columns=['total']
trans=trans.reset_index()
trans

,feature,total
0,account,0
1,admin fee,-1
2,consent,1
3,consultation,7
4,customer service,3
5,data,0
6,doctor,12
7,download invoice,1
8,edit review,-1
9,information,5


In [7]:
feature_counted = feature_counted.merge(trans, on='feature',how='inner')
feature_counted['max'] = feature_counted['count']*2
feature_counted['sentiment_score'] = feature_counted['total']/feature_counted['max']
feature_counted['s_x_v'] = feature_counted['sentiment_score']*feature_counted['count']
feature_counted

,feature,count,total,max,sentiment_score,s_x_v
0,user experience,39,49,78,0.628205,24.5
1,doctor,17,12,34,0.352941,6.0
2,response,16,15,32,0.468750,7.5
3,consultation,10,7,20,0.350000,3.5
4,information,6,5,12,0.416667,2.5
5,customer service,4,3,8,0.375000,1.5
6,payment,4,0,8,0.000000,0.0
7,data,3,0,6,0.000000,0.0
8,account,2,0,4,0.000000,0.0
9,medicine,2,2,4,0.500000,1.0


In [8]:
swot2 = pd.read_csv(r'features_halodoc.csv')
swot2

,score,content_contractions,compound,neg,neu,pos,vader_rating,keyword,word,feature
0,1,"i have already uploaded the medicine recipe, w...",0.0000,0.0,1.000,0.000,0,"[('medicine recipe', 0.6271), ('buying medicin...","('medicine recipe', 0.6271)",prescription
1,1,"i have already uploaded the medicine recipe, w...",0.0000,0.0,1.000,0.000,0,"[('medicine recipe', 0.6271), ('buying medicin...","'buying medicine', 0.5984)",buying medicine
2,1,1. Cannot remove profile 2. Cannot delete account,0.0000,0.0,1.000,0.000,0,"[('remove profile', 0.7197), ('delete account'...","('remove profile', 0.7197)",delete account
3,3,"Doctor consultation always slow in responding,...",0.0000,0.0,1.000,0.000,0,"[('doctor consultation', 0.6149), ('has insura...","('doctor consultation', 0.6149)",consultation
4,3,"Doctor consultation always slow in responding,...",0.0000,0.0,1.000,0.000,0,"[('doctor consultation', 0.6149), ('has insura...","'has insurance integration', 0.4072)",insurance integration
...,...,...,...,...,...,...,...,...,...,...
117,5,Keep a good work and uphrade for more best exp...,0.8122,0.0,0.520,0.480,1,"[('uphrade', 0.5092), ('experience', 0.3705), ...","'experience', 0.3705)",user experience
118,5,"Helpful, especially for covid patient",0.4215,0.0,0.588,0.412,1,"[('patient', 0.3986)]","('patient', 0.3986)",user experience
119,4,Helping many things of getting better solution...,0.7506,0.0,0.519,0.481,1,"[('doctor', 0.4301), ('solution', 0.3033), ('a...","('doctor', 0.4301)",doctor
120,4,Helping many things of getting better solution...,0.7506,0.0,0.519,0.481,1,"[('doctor', 0.4301), ('solution', 0.3033), ('a...","'solution', 0.3033)",solution


In [16]:
feature_counted2 = swot2['feature'].value_counts()
feature_counted2 = pd.DataFrame(feature_counted2)
feature_counted2.reset_index(inplace=True)
feature_counted2 = feature_counted2.rename(columns={'index':'feature','feature':'count'})
feature_counted2

,feature,count
0,user experience,37
1,doctor,14
2,response,9
3,buying medicine,9
4,consultation,5
5,service,4
6,customer service,3
7,chat,3
8,prescription,3
9,time,2


In [17]:
feature_counted2['count'].sum()

122

In [18]:
trans2=swot2.groupby('feature')['vader_rating'].sum()
trans2=pd.DataFrame(trans2)
trans2.columns=['total']
trans2=trans2.reset_index()
trans2

,feature,total
0,app update,1
1,appointment history,1
2,bug,-1
3,buying medicine,5
4,chat,2
5,consultation,5
6,customer service,-3
7,delete account,0
8,delivery,1
9,delivery fee,1


In [19]:
feature_counted2 = feature_counted2.merge(trans2, on='feature',how='inner')
feature_counted2['max'] = feature_counted2['count']*2
feature_counted2['sentiment_score'] = feature_counted2['total']/feature_counted2['max']
feature_counted2['s_x_v'] = feature_counted2['sentiment_score']*feature_counted2['count']
feature_counted2

,feature,count,total,max,sentiment_score,s_x_v
0,user experience,37,45,74,0.608108,22.5
1,doctor,14,14,28,0.500000,7.0
2,response,9,12,18,0.666667,6.0
3,buying medicine,9,5,18,0.277778,2.5
4,consultation,5,5,10,0.500000,2.5
5,service,4,7,8,0.875000,3.5
6,customer service,3,-3,6,-0.500000,-1.5
7,chat,3,2,6,0.333333,1.0
8,prescription,3,0,6,0.000000,0.0
9,time,2,0,4,0.000000,0.0


In [57]:
halodoc = feature_counted2[['feature','s_x_v']]
halodoc = halodoc.rename(columns={'s_x_v':'s_x_v_halodoc'})
alodokter = feature_counted[['feature','s_x_v']]
alodokter = alodokter.rename(columns={'s_x_v':'s_x_v_alodokter'})
fitur = halodoc.merge(alodokter, on='feature',how='outer')
fitur = fitur.fillna(0)
fitur['total'] = abs(fitur['s_x_v_halodoc']) + abs(fitur['s_x_v_alodokter'])
fitur

,feature,s_x_v_halodoc,s_x_v_alodokter,total
0,user experience,22.5,24.5,47.0
1,doctor,7.0,6.0,13.0
2,response,6.0,7.5,13.5
3,buying medicine,2.5,0.0,2.5
4,consultation,2.5,3.5,6.0
5,service,3.5,0.0,3.5
6,customer service,-1.5,1.5,3.0
7,chat,1.0,0.0,1.0
8,prescription,0.0,0.5,0.5
9,time,0.0,0.0,0.0


In [61]:
def ngitung(a,b):
    if (b==0):
        return 0
    else:
        return a/b
fitur['fps_halodoc']=''
fitur['fps_alodokter']=''
for i in range (len(fitur['feature'])):
    fitur['fps_halodoc'][i] = ngitung(fitur['s_x_v_halodoc'][i],fitur['total'][i])
    fitur['fps_alodokter'][i] = ngitung(fitur['s_x_v_alodokter'][i],fitur['total'][i])
fitur

<ipython-input-61-ddb7dd1809e3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fitur['fps_halodoc'][i] = ngitung(fitur['s_x_v_halodoc'][i],fitur['total'][i])
<ipython-input-61-ddb7dd1809e3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fitur['fps_alodokter'][i] = ngitung(fitur['s_x_v_alodokter'][i],fitur['total'][i])


,feature,s_x_v_halodoc,s_x_v_alodokter,total,fps_halodoc,fps_alodokter
0,user experience,22.5,24.5,47.0,0.478723,0.521277
1,doctor,7.0,6.0,13.0,0.538462,0.461538
2,response,6.0,7.5,13.5,0.444444,0.555556
3,buying medicine,2.5,0.0,2.5,1.000000,0.000000
4,consultation,2.5,3.5,6.0,0.416667,0.583333
5,service,3.5,0.0,3.5,1.000000,0.000000
6,customer service,-1.5,1.5,3.0,-0.500000,0.500000
7,chat,1.0,0.0,1.0,1.000000,0.000000
8,prescription,0.0,0.5,0.5,0.000000,1.000000
9,time,0.0,0.0,0.0,0,0


In [64]:
def sentimen (a):
    if a >- 0.1:
        return ("positive")
    elif a <= (-0.1):
        return("negative")
    else:
        return ("neutral")
fitur['sentiment_halodoc'] = fitur['fps_halodoc'].apply(lambda x: sentimen(x))
fitur['sentiment_alodokter'] = fitur['fps_alodokter'].apply(lambda x: sentimen(x))
fitur

,feature,s_x_v_halodoc,s_x_v_alodokter,total,fps_halodoc,fps_alodokter,sentiment_halodoc,sentiment_alodokter
0,user experience,22.5,24.5,47.0,0.478723,0.521277,positive,positive
1,doctor,7.0,6.0,13.0,0.538462,0.461538,positive,positive
2,response,6.0,7.5,13.5,0.444444,0.555556,positive,positive
3,buying medicine,2.5,0.0,2.5,1.000000,0.000000,positive,positive
4,consultation,2.5,3.5,6.0,0.416667,0.583333,positive,positive
5,service,3.5,0.0,3.5,1.000000,0.000000,positive,positive
6,customer service,-1.5,1.5,3.0,-0.500000,0.500000,negative,positive
7,chat,1.0,0.0,1.0,1.000000,0.000000,positive,positive
8,prescription,0.0,0.5,0.5,0.000000,1.000000,positive,positive
9,time,0.0,0.0,0.0,0,0,positive,positive


In [74]:
fitur['fps_tes'] = ''
for i in range (len(fitur['feature'])):
    fitur['fps_tes'][i] = random.uniform(-1,1)
fitur

<ipython-input-74-7746daf66c63>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fitur['fps_tes'][i] = random.uniform(-1,1)


,feature,s_x_v_halodoc,s_x_v_alodokter,total,fps_halodoc,fps_alodokter,sentiment_halodoc,sentiment_alodokter,avg_fps,uniqueness_halodoc,uniqueness_alodokter,fps_tes
0,user experience,22.5,24.5,47.0,0.478723,0.521277,positive,positive,0.5,None,None,-0.745987
1,doctor,7.0,6.0,13.0,0.538462,0.461538,positive,positive,0.5,None,None,-0.116178
2,response,6.0,7.5,13.5,0.444444,0.555556,positive,positive,0.5,None,None,-0.443347
3,buying medicine,2.5,0.0,2.5,1.000000,0.000000,positive,positive,0.5,above market,below market,0.454525
4,consultation,2.5,3.5,6.0,0.416667,0.583333,positive,positive,0.5,None,None,-0.613559
5,service,3.5,0.0,3.5,1.000000,0.000000,positive,positive,0.5,above market,below market,-0.628361
6,customer service,-1.5,1.5,3.0,-0.500000,0.500000,negative,positive,0.0,below market,above market,-0.667176
7,chat,1.0,0.0,1.0,1.000000,0.000000,positive,positive,0.5,above market,below market,0.267542
8,prescription,0.0,0.5,0.5,0.000000,1.000000,positive,positive,0.5,below market,above market,0.557448
9,time,0.0,0.0,0.0,0,0,positive,positive,0.0,None,None,0.358919


In [75]:
fitur['avg_fps2'] = (fitur['fps_halodoc'] + fitur['fps_alodokter']+fitur['fps_tes'])/2
fitur

,feature,s_x_v_halodoc,s_x_v_alodokter,total,fps_halodoc,fps_alodokter,sentiment_halodoc,sentiment_alodokter,avg_fps,uniqueness_halodoc,uniqueness_alodokter,fps_tes,avg_fps2
0,user experience,22.5,24.5,47.0,0.478723,0.521277,positive,positive,0.5,None,None,-0.745987,0.127006
1,doctor,7.0,6.0,13.0,0.538462,0.461538,positive,positive,0.5,None,None,-0.116178,0.441911
2,response,6.0,7.5,13.5,0.444444,0.555556,positive,positive,0.5,None,None,-0.443347,0.278326
3,buying medicine,2.5,0.0,2.5,1.000000,0.000000,positive,positive,0.5,above market,below market,0.454525,0.727262
4,consultation,2.5,3.5,6.0,0.416667,0.583333,positive,positive,0.5,None,None,-0.613559,0.193221
5,service,3.5,0.0,3.5,1.000000,0.000000,positive,positive,0.5,above market,below market,-0.628361,0.185820
6,customer service,-1.5,1.5,3.0,-0.500000,0.500000,negative,positive,0.0,below market,above market,-0.667176,-0.333588
7,chat,1.0,0.0,1.0,1.000000,0.000000,positive,positive,0.5,above market,below market,0.267542,0.633771
8,prescription,0.0,0.5,0.5,0.000000,1.000000,positive,positive,0.5,below market,above market,0.557448,0.778724
9,time,0.0,0.0,0.0,0,0,positive,positive,0.0,None,None,0.358919,0.179460


In [76]:
def unq(a,b):
    if (a-b) >= 0.1:
        return "above market"
    elif (a-b <= -0.1):
        return "below market"
    else:
        return "unique"


fitur['uniqueness_halodoc2'] = fitur.apply(lambda x: unq(x.fps_halodoc,x.avg_fps2),axis=1)
fitur['uniqueness_alodokter2'] = fitur.apply(lambda x: unq(x.fps_alodokter,x.avg_fps2),axis=1)
fitur

,feature,s_x_v_halodoc,s_x_v_alodokter,total,fps_halodoc,fps_alodokter,sentiment_halodoc,sentiment_alodokter,avg_fps,uniqueness_halodoc,uniqueness_alodokter,fps_tes,avg_fps2,uniqueness_halodoc2,uniqueness_alodokter2
0,user experience,22.5,24.5,47.0,0.478723,0.521277,positive,positive,0.5,None,None,-0.745987,0.127006,above market,above market
1,doctor,7.0,6.0,13.0,0.538462,0.461538,positive,positive,0.5,None,None,-0.116178,0.441911,unique,unique
2,response,6.0,7.5,13.5,0.444444,0.555556,positive,positive,0.5,None,None,-0.443347,0.278326,above market,above market
3,buying medicine,2.5,0.0,2.5,1.000000,0.000000,positive,positive,0.5,above market,below market,0.454525,0.727262,above market,below market
4,consultation,2.5,3.5,6.0,0.416667,0.583333,positive,positive,0.5,None,None,-0.613559,0.193221,above market,above market
5,service,3.5,0.0,3.5,1.000000,0.000000,positive,positive,0.5,above market,below market,-0.628361,0.185820,above market,below market
6,customer service,-1.5,1.5,3.0,-0.500000,0.500000,negative,positive,0.0,below market,above market,-0.667176,-0.333588,below market,above market
7,chat,1.0,0.0,1.0,1.000000,0.000000,positive,positive,0.5,above market,below market,0.267542,0.633771,above market,below market
8,prescription,0.0,0.5,0.5,0.000000,1.000000,positive,positive,0.5,below market,above market,0.557448,0.778724,below market,above market
9,time,0.0,0.0,0.0,0,0,positive,positive,0.0,None,None,0.358919,0.179460,below market,below market


# EVALUATION